In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
from pydrake.all import StartMeshcat

# if you're not launching jupyter from the repo root, fix the path:
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

meshcat = StartMeshcat()
print("Meshcat:", meshcat.web_url())

INFO:drake:Meshcat listening for connections at http://localhost:7000


Meshcat: http://localhost:7000


In [21]:
import os
import numpy as np
from pydrake.all import StartMeshcat
from pydrake.multibody.tree import RevoluteJoint
from rocky.build import build_robot_diagram_two
from rocky.ilqr import ILQRController, CostParams
from rocky.dynamics import ArmParams, forward_kinematics

rng = np.random.default_rng()
enemy_config_mode = "default"  # options: "random", "default", "custom"
custom_enemy_degs = [30.0, -100.0]


# --- URDF ---
urdf = "../rocky.urdf"

# --- Build ---
bundle = build_robot_diagram_two(
    urdf_path=urdf,
    time_step=1e-3,
    gravity_vec=(0., 0., 0.),
    meshcat=meshcat,
)

plant, ctx, sim = bundle.plant, bundle.plant_context, bundle.simulator
ally, enemy = bundle.ally, bundle.enemy

W = plant.world_frame()
enemy_target_frame = plant.GetFrameByName("enemy_target")
ally_wall_frame = plant.GetFrameByName("wall", model_instance=ally)
ally_glove_frame = plant.GetFrameByName("glove", model_instance=ally)
enemy_wall_frame = plant.GetFrameByName("wall", model_instance=enemy)
enemy_elbow_frame = plant.GetFrameByName("elbow_marker", model_instance=enemy)
enemy_glove_frame = plant.GetFrameByName("glove", model_instance=enemy)


# --- Joint helpers ---
def get_revolute_joints_for_instance(plant, instance):
    joints = [plant.get_joint(j) for j in plant.GetJointIndices(instance)]
    return [j for j in joints if isinstance(j, RevoluteJoint)]


def set_angles_for_instance(plant, ctx, instance, angles_rad):
    revs = get_revolute_joints_for_instance(plant, instance)
    for j, q in zip(revs, angles_rad):
        j.set_angle(ctx, float(q))


def set_zero_torques(plant, ctx):
    plant.get_actuation_input_port().FixValue(ctx, np.zeros(plant.num_actuators()))


def get_ally_state(plant, ctx, ally):
    q = plant.GetPositions(ctx, ally)[:2]
    dq = plant.GetVelocities(ctx, ally)[:2]
    return np.hstack([q, dq])


def get_ally_shoulder_in_world(plant, ctx, ally_wall_frame):
    X_W_wall = plant.CalcRelativeTransform(ctx, W, ally_wall_frame)
    shoulder_in_wall = np.array([0.0, 0.0, 1.0])   # joint origin in wall frame
    return X_W_wall.translation() + shoulder_in_wall


def get_target_pos_2d(plant, ctx):
    p_W_shoulder = get_ally_shoulder_in_world(plant, ctx, ally_wall_frame)
    p_W_T = plant.CalcRelativeTransform(ctx, W, enemy_target_frame).translation()
    rel = p_W_T - p_W_shoulder              # [dx, dy, dz] in world
    return np.array([rel[0], rel[2]])       # model x,z  (no sign flip)


def get_enemy_pos_2d(plant, ctx):
    p_W_shoulder = get_ally_shoulder_in_world(plant, ctx, ally_wall_frame)
    p_W_EE = plant.CalcRelativeTransform(ctx, W, enemy_glove_frame).translation()
    rel = p_W_EE - p_W_shoulder
    return np.array([rel[0], rel[2]])       # model x,z  (no sign flip)


def get_ally_pos_2d_from_model(x, params):
    return forward_kinematics(x[:2], params)


def apply_ally_torque(plant, ctx, ally, u2):
    tau = np.zeros(plant.num_actuators())
    idx = list(plant.GetJointActuatorIndices(ally))
    tau[idx[0]] = u2[0]
    tau[idx[1]] = u2[1]
    plant.get_actuation_input_port().FixValue(ctx, tau)


def randomize_enemy_configuration(
    clearance_buffer=0.02,
    clearance_target=None,
    clearance_ally=None,
    max_tries=200,
):
    """Sample random enemy joint angles that keep the glove away from target/ally."""
    glove_radius = 0.05
    target_radius = 0.03
    clearance_target = clearance_target or (glove_radius + target_radius + clearance_buffer)
    clearance_ally = clearance_ally or (2 * glove_radius + clearance_buffer)

    joints = get_revolute_joints_for_instance(plant, enemy)
    limits = []
    for j in joints:
        lo = j.position_lower_limit()
        hi = j.position_upper_limit()
        if not np.isfinite(lo):
            lo = -np.pi
        if not np.isfinite(hi):
            hi = np.pi
        limits.append((lo, hi))

    for attempt in range(1, max_tries + 1):
        candidate = np.array([rng.uniform(lo, hi) for lo, hi in limits])
        set_angles_for_instance(plant, ctx, enemy, candidate)

        p_enemy = plant.CalcRelativeTransform(ctx, W, enemy_glove_frame).translation()
        p_target = plant.CalcRelativeTransform(ctx, W, enemy_target_frame).translation()
        p_ally = plant.CalcRelativeTransform(ctx, W, ally_glove_frame).translation()
        p_elbow = plant.CalcRelativeTransform(ctx, W, enemy_elbow_frame).translation()
        wall_x = plant.CalcRelativeTransform(ctx, W, enemy_wall_frame).translation()[0]

        if not (p_elbow[0] < wall_x and p_enemy[0] < wall_x):
            continue
        if np.linalg.norm(p_enemy - p_target) < clearance_target:
            continue
        if np.linalg.norm(p_enemy - p_ally) < clearance_ally:
            continue
        print(f"Enemy randomized in {attempt} tries.")
        return candidate

    raise RuntimeError("Failed to sample a collision-free enemy configuration")


params = ArmParams()
cost = CostParams(
    Q=np.diag([2.0, 2.0, 0.1, 0.1]),
    R=0.01 * np.eye(2),
    Qf=np.diag([50.0, 50.0, 1.0, 1.0]),
    target_weight=425.0,
    r_min=0.01, # Turning off Exponential Cost at a certain distance
    avoid_weight=30.0,
    avoid_sigma=0.08,
)

dt_mpc = 0.02  # CHANGED: 0.05 is too large for stable feedback on this arm
horizon = 50   # Adjusted horizon to match new dt (40 * 0.02 = 0.8s lookahead)

ctrl = ILQRController(
    horizon=horizon,
    dt=dt_mpc,
    params=params,
    cost=cost,
    verbose=True,
)

print(ctrl)

# Ally starts from a fixed configuration for repeatability
ally_angles = np.deg2rad([45.0, -130.0])
set_angles_for_instance(plant, ctx, ally, ally_angles)

# Enemy configuration selection
if enemy_config_mode == "random":
    enemy_angles = randomize_enemy_configuration()
elif enemy_config_mode == "default":
    enemy_angles = np.deg2rad([45.0, -140.0])
elif enemy_config_mode == "custom":
    enemy_angles = np.deg2rad(custom_enemy_degs)
else:
    raise ValueError(f"Unknown enemy_config_mode: {enemy_config_mode}")
print("Enemy start angles (deg):", np.rad2deg(enemy_angles))
set_angles_for_instance(plant, ctx, enemy, enemy_angles)

sim.AdvanceTo(0.0)
print("Done.")

ILQRController(
  horizon=50, dt=0.02, max_iter=60, tol=0.001, reg=0.0001, verbose=True
  params: l1=0.5, l2=0.5, m1=0.6, m2=0.5, I1=0.002, I2=0.002, damping=[0.100 0.100], gravity=0, torque_limit=[50.000 50.000]
  cost: Q=[[2.000 0.000 0.000 0.000]
 [0.000 2.000 0.000 0.000]
 [0.000 0.000 0.100 0.000]
 [0.000 0.000 0.000 0.100]], R=[[0.010 0.000]
 [0.000 0.010]], Qf=[[50.000  0.000  0.000  0.000]
 [ 0.000 50.000  0.000  0.000]
 [ 0.000  0.000  1.000  0.000]
 [ 0.000  0.000  0.000  1.000]], target_w=425.0, avoid_w=30.0, avoid_sigma=0.08, x_goal=[0.000 0.000 0.000 0.000]
  alphas=[1.0, 0.5, 0.25, 0.1, 0.05]
)
Enemy start angles (deg): [  45. -140.]
Done.


In [22]:
import time

t_final = 10.0
dt_ctrl = ctrl.dt
base_avoid_weight = cost.avoid_weight
avoid_decay_time = 5.0  # seconds to significantly reduce avoidance weight
min_avoid_scale = 0.1

t = sim.get_context().get_time()
u_traj_prev = None
step = 0
mode = "mpc"  # start in MPC mode

print("Starting MPC loop...")

def has_reached_target(ally_pos, target_pos, x,
                       R_target=0.1,
                       R_ally=0.005,
                       vel_tol=0.1):

    # distance between centers in task space
    pos_err = np.linalg.norm(ally_pos - target_pos)

    # geometric contact threshold
    contact_dist = R_target + R_ally

    return pos_err <= contact_dist


while t < t_final:
    # 1) Current state
    x_current = get_ally_state(plant, ctx, ally)

    # 2) Positions
    target_pos_2d = get_target_pos_2d(plant, ctx)
    enemy_pos_2d  = get_enemy_pos_2d(plant, ctx)
    ally_pos_2d   = get_ally_pos_2d_from_model(x_current, params)

    print("target positions: ", target_pos_2d)
    print("enemy positions: ", enemy_pos_2d)
    print("ally positions: ", ally_pos_2d)

    # 3) Check for NaN in state (Simulation exploded)
    if np.any(np.isnan(x_current)):
        print("ERROR: Simulation state is NaN. Physics exploded.")
        break

    # 4) Check target hit and switch to passive mode (one-way)
    if mode == "mpc" and has_reached_target(ally_pos_2d, target_pos_2d, x_current):
        print(f"\n*** Target reached at t={t:.2f}s ***")
        print(f"ally EE pos: {ally_pos_2d}, target: {target_pos_2d}")
        print("*** Target reached; switching to passive mode ***")
        mode = "passive"

    # 5) Control law: MPC vs passive. Can change this to 'attack' / 'defense' next
    if mode == "mpc":
        # Decay enemy-avoidance weight over time
        decay = np.exp(-t / avoid_decay_time)
        avoid_scale = max(min_avoid_scale, decay)
        ctrl.cost.avoid_weight = base_avoid_weight * avoid_scale

        # MPC step
        x_pred, result = ctrl.step_mpc(
            x_current,
            target_pos=target_pos_2d,
            enemy_pos=enemy_pos_2d,
            prev_u_trj=u_traj_prev,
        )

        if np.isfinite(result.cost) and result.converged:
            u_traj_prev = result.u_trj
            u0 = result.u_trj[0]
        else:
            print(f"[Warning] iLQR failed (cost: {result.cost}, conv: {result.converged}). Braking.")
            # Damping torque to stop robot if controller fails
            u0 = params.clip_u(-2.0 * x_current[2:])
            u_traj_prev = None  # Reset warm start
    else:
        # Passive after hit: zero (or damping) torques
        # You can replace np.zeros(2) by a damping law if you want.

        # change here or new mode ==
        u0 = np.zeros(2)


    # 6) Debug prints
    if step % 5 == 0:
        if mode == "mpc":
            dbg_cost = result.cost
            dbg_avoid = ctrl.cost.avoid_weight
        else:
            dbg_cost = float("nan")
            dbg_avoid = ctrl.cost.avoid_weight  # last value from MPC phase

        print(
            f"t={t:.3f} | mode={mode} | "
            f"x={np.array2string(x_current, precision=2)} | "
            f"cost={dbg_cost:.1f} | avoid_w={dbg_avoid:.1f} | "
            f"u0={np.array2string(u0, precision=2)}"
        )

    # 7) Apply control and advance sim
    apply_ally_torque(plant, ctx, ally, u0)
    sim.AdvanceTo(t + dt_ctrl)
    t = sim.get_context().get_time()
    step += 1

print("MPC run complete.")


Starting MPC loop...
target positions:  [0.8  0.25]
enemy positions:  [0.59002448 0.14454396]
ally positions:  [0.39713126 0.14454396]
[iLQR] init cost 2395.4612
[iLQR] iter 0 alpha 0.50 -> cost 2051.7595
[iLQR] converged at iter 0 cost 2051.7595
[MPC] x=[ 0.78539816 -2.26892803  0.          0.        ], target=[0.8  0.25], u0=[-12.37426572   5.55105491], cost=2051.7595330834947
t=0.000 | mode=mpc | x=[ 0.79 -2.27  0.    0.  ] | cost=2051.8 | avoid_w=30.0 | u0=[-12.37   5.55]
target positions:  [0.8  0.25]
enemy positions:  [0.59002448 0.14454396]
ally positions:  [0.40638602 0.15364505]
[iLQR] init cost 2516.2452
[iLQR] iter 0 alpha 1.00 -> cost 885.1196
[iLQR] converged at iter 0 cost 885.1196
[MPC] x=[ 0.75989127 -2.24271303 -2.52658584  2.63861483], target=[0.8  0.25], u0=[-24.41890324   8.46379453], cost=885.1195527027725
target positions:  [0.8  0.25]
enemy positions:  [0.59002448 0.14454396]
ally positions:  [0.43976586 0.19254722]
[iLQR] init cost 747.8707
[iLQR] iter 0 alpha 0

/home/para/RoboRocky/rocky/ilqr.py:135: RuntimeWarning: overflow encountered in matmul
  l += 0.5 * dx.T @ cost.Q @ dx + 0.5 * u.T @ cost.R @ u
/home/para/RoboRocky/rocky/dynamics.py:94: RuntimeWarning: overflow encountered in matmul
  ddq = np.linalg.solve(M, u_clipped - C @ dq - G - damping)


target positions:  [0.8  0.25]
enemy positions:  [0.59002448 0.14454396]
ally positions:  [0.50398368 0.24668885]
[iLQR] init cost 625.0653
[iLQR] iter 0 alpha 1.00 -> cost 192.9100
[iLQR] converged at iter 0 cost 192.9100
[MPC] x=[ 0.51986432 -1.95011765 -6.56674525 11.10523275], target=[0.8  0.25], u0=[20.87539406  4.75522724], cost=192.9100344937536
target positions:  [0.8  0.25]
enemy positions:  [0.61939189 0.1502261 ]
ally positions:  [0.58234003 0.2727925 ]
[iLQR] init cost 1980.1627
[iLQR] iter 0 alpha 0.50 -> cost 1891.1488
[iLQR] converged at iter 0 cost 1891.1488
[MPC] x=[ 0.43421537 -1.74459869 -2.31931253  8.93444117], target=[0.8  0.25], u0=[ -4.5244577  -10.39346142], cost=1891.1487750993892
target positions:  [0.8  0.25]
enemy positions:  [0.67889078 0.15483119]
ally positions:  [0.62538585 0.28127198]
[iLQR] init cost 2430.3239
[iLQR] iter 0 alpha 0.50 -> cost 1436.1925
[iLQR] converged at iter 0 cost 1436.1925
[MPC] x=[ 0.39254216 -1.63038881 -1.76865081  2.73606877],

KeyboardInterrupt: 